# Model 04

Prediction of claim labels based on the matched evidence.

Ref:
- [Hugging face pre-trained models](https://huggingface.co/transformers/v3.3.1/pretrained_models.html)
- [Hugging face guide to fine-tuning](https://huggingface.co/transformers/v3.3.1/custom_datasets.html)
- [Hugging face guide to fine-tuning easy](https://huggingface.co/docs/transformers/training)
- [SO Guide](https://stackoverflow.com/a/64156912)
- [roberta-large-mnli](https://github.com/facebookresearch/fairseq/tree/main/examples/roberta)

## Setup

### Working Directory

In [1]:
# Change the working directory to project root
from pathlib import Path
import os
ROOT_DIR = Path.cwd()
while not ROOT_DIR.joinpath("src").exists():
    ROOT_DIR = ROOT_DIR.parent
os.chdir(ROOT_DIR)

### File paths

In [2]:
MODEL_PATH = ROOT_DIR.joinpath("./result/models/*")
DATA_PATH = ROOT_DIR.joinpath("./data/*")
NER_PATH = ROOT_DIR.joinpath("./result/ner/*")

### Dependencies

In [3]:
# Imports and dependencies
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import Linear, Module, CrossEntropyLoss, Dropout
from transformers import RobertaModel, RobertaTokenizer
from torch.optim import Adam
from torch.optim.lr_scheduler import LinearLR
from torch.nn.functional import relu, softmax
from torcheval.metrics import MulticlassAccuracy, MulticlassF1Score

from src.torch_utils import get_torch_device
import json
from dataclasses import dataclass
from typing import List, Union, Tuple
from tqdm import tqdm
import random
import numpy as np
from datetime import datetime
from math import exp

TORCH_DEVICE = get_torch_device()

/opt/homebrew/Caskroom/miniconda/base/envs/comp90042_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch device is 'mps'


## Dataset

In [4]:
@dataclass
class ClaimEvidencePair:
    claim_id:str
    evidence_id:str
    label:int = 0

In [5]:
class SiameseLabelDataset(Dataset):
    
    # Pre-trained model has:
    # label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
    LABEL_MAP = {
        "REFUTES": [0],
        "NOT_ENOUGH_INFO": [1],
        "SUPPORTS": [2],
        "DISPUTED": [0, 2],
    }
    
    def __init__(
        self,
        claims_paths:List[Path],
        evidence_path:Path,
        device = None,
        verbose:bool=True
    ) -> None:
        super(SiameseLabelDataset, self).__init__()
        self.verbose = verbose
        self.device = device

        # Load claims data from json, this is a list as we could use
        # multiple json files in the same dataset
        self.claims = dict()
        for json_file in claims_paths:
            with open(json_file, mode="r") as f:
                self.claims.update(json.load(fp=f))
        
        # Load evidence library
        self.evidence = dict()
        with open(evidence_path, mode="r") as f:
            self.evidence.update(json.load(fp=f))
        
        # Generate the data
        self.data = self.__generate_data()
        return

    def __generate_data(self):
        print("Generate siamese dataset")
        
        data = []
        for claim_id, claim in tqdm(
            iterable=self.claims.items(),
            desc="claims",
            disable=not self.verbose
        ):
            # Get evidence ids associated with each claim
            evidence_ids = claim["evidences"]
            
            # Get the claim label
            label = claim["claim_label"]
            
            # Encode the label
            label_encodings = self.LABEL_MAP.get(label, 99)
            
            if 99 in label_encodings:
                print(f"Invalid label {label} for {claim_id}")
                
            for label_encoding in label_encodings:
                
                # Assign the labels to each claim/evidence pairs
                for evidence_id in evidence_ids:
                    data.append(ClaimEvidencePair(
                        claim_id=claim_id,
                        evidence_id=evidence_id,
                        label=label_encoding
                    ))
            
            continue
        
        print(f"Generated data n={len(data)}")
        
        return data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx) -> Tuple[Union[str, torch.Tensor]]:
        # Fetch the required data rows
        data = self.data[idx]
        
        # Get the label
        label = torch.tensor(data.label, device=self.device)
        # label = torch.unsqueeze(label, -1)
        
        # Get text ids
        claim_id = data.claim_id
        evidence_id = data.evidence_id
        
        # Get text
        claim_text = self.claims[claim_id]["claim_text"]
        evidence_text = self.evidence[evidence_id]

        return (claim_text, evidence_text, label)

## Build model

In [6]:
class SiameseClassifierRoberta(Module):
    
    def __init__(
            self,
            pretrained_name:str,
            device,
            **kwargs
        ) -> None:
        super(SiameseClassifierRoberta, self).__init__(**kwargs)
        self.device = device
        
        # Use a pretrained tokenizer
        self.tokenizer = RobertaTokenizer.from_pretrained(pretrained_name)
        
        # Use a pretrained model
        self.bert = RobertaModel.from_pretrained(pretrained_name)
        self.bert.to(device=device)
        
        # Classification layers
        self.linear1 = Linear(768, 256, bias=True, device=device)
        self.linear2 = Linear(256, 3, bias=True, device=device)
        self.relu = relu
        self.softmax = softmax
        self.dropout_in = Dropout(p=0.2)
        self.dropout_out = Dropout(p=0.5)
        return
        
    def forward(self, claim_texts, evidence_texts) -> Tuple[torch.Tensor]:
        
        # Run the tokenizer
        t_kwargs = {
            "return_tensors": "pt",
            "padding": True,
            "truncation": True,
            "max_length": 512,
            "add_special_tokens":True
        }
        # Use a cross-encoding pattern
        x = self.tokenizer(claim_texts, evidence_texts, **t_kwargs)
        x = x["input_ids"].to(device=self.device)
        x = self.bert(x, return_dict=True).pooler_output
        # dim=768
        
        # Run classification layers
        x = self.dropout_in(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout_out(x)
        x = self.linear2(x)
        
        # Create the predictions
        y = self.softmax(x, dim=1)
        
        return y

## Training and evaluation loop

In [7]:
CHECKPOINT_NAME = f"model_04_roberta_base_3_class.pth"
with open(MODEL_PATH.with_name(CHECKPOINT_NAME), mode="rb") as f:
    model = torch.load(f)

In [8]:
# model = SiameseClassifierRoberta(
#     pretrained_name="roberta-base",
#     device=TORCH_DEVICE
# )

In [9]:
loss_fn = CrossEntropyLoss()
optimizer = Adam(
    params=model.parameters(),
    lr=0.000002
) #! Hyperparams

In [10]:
run_time = datetime.now().strftime('%Y_%m_%d_%H_%M')
# MODEL_NAME = f"model_04_roberta_base_{run_time}.pth"
MODEL_NAME = CHECKPOINT_NAME
N_EPOCHS = 100
BATCH_SIZE = 24


In [11]:
train_data = SiameseLabelDataset(
    claims_paths=[DATA_PATH.with_name("train-claims.json")],
    evidence_path=DATA_PATH.with_name("evidence.json"),
    device=TORCH_DEVICE,
)

train_dataloader = DataLoader(
    dataset=train_data,
    shuffle=True,
    batch_size=BATCH_SIZE
)

Generate siamese dataset


claims: 100%|██████████| 1228/1228 [00:00<00:00, 566685.59it/s]

Generated data n=4514


In [12]:
dev_data = SiameseLabelDataset(
    claims_paths=[DATA_PATH.with_name("dev-claims.json")],
    evidence_path=DATA_PATH.with_name("evidence.json"),
    device=TORCH_DEVICE,
)

dev_dataloader = DataLoader(
    dataset=dev_data,
    shuffle=False,
    batch_size=BATCH_SIZE
)

Generate siamese dataset


claims: 100%|██████████| 154/154 [00:00<00:00, 594772.39it/s]

Generated data n=549


In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
metric_accuracy = MulticlassAccuracy()
metric_f1 = MulticlassF1Score()

scheduler = LinearLR(
    optimizer=optimizer,
    start_factor=0.1,
    end_factor=1,
    total_iters=int(N_EPOCHS/10),
    verbose=True
)
last_epoch_loss = 999
for epoch in range(N_EPOCHS):
    
    print(f"Epoch: {epoch} of {N_EPOCHS}\n")
    
    # Run training
    model.train()
    
    train_batches = tqdm(train_dataloader, desc="train batches")
    running_losses = []
    for batch in train_batches:
        claim_texts, evidence_texts, labels = batch
        
        # Reset optimizer
        optimizer.zero_grad()
        
        # Forward + loss
        predictions = model(claim_texts, evidence_texts)
        loss = loss_fn(predictions, labels)
        
        # Backward + optimiser
        loss.backward()
        optimizer.step()
        
        # Update running loss
        batch_loss = loss.item() * len(batch)
        running_losses.append(batch_loss)
        
        train_batches.postfix = f"loss: {batch_loss:.3f}"
        
        continue
    
    scheduler.step()
    
    epoch_loss = np.average(running_losses)
    print(f"Average epoch loss: {epoch_loss}")
    
    # Save model
    if epoch_loss <= last_epoch_loss:
        torch.save(model, MODEL_PATH.with_name(MODEL_NAME))
        print(f"Saved model to: {MODEL_PATH.with_name(MODEL_NAME)}")
    last_epoch_loss = epoch_loss
    
    # Evaluate every 5 epochs
    # if epoch % 5 != 0:
    #     continue
    
    # Run evaluation
    model.eval()
    
    dev_batches = tqdm(dev_dataloader, desc="dev batches")
    dev_acc = []
    dev_f1 = []
    for batch in dev_batches:
        claim_texts, evidence_texts, labels = batch
        
        # Forward
        predictions = model(claim_texts, evidence_texts)
        
        # Prediction
        predicted = torch.argmax(predictions, dim=1)
        
        # Metrics
        metric_accuracy.update(predicted.cpu(), labels.cpu())
        metric_f1.update(predicted.cpu(), labels.cpu())
        
        acc = metric_accuracy.compute()
        f1 = metric_f1.compute()
        
        dev_acc.append(acc)
        dev_f1.append(f1)
        
        dev_batches.postfix = f" acc: {acc:.3f} f1: {f1:.3f}"
        
        continue
    
    val_acc = np.mean(dev_acc)
    val_f1 = np.mean(dev_f1)
    
    print(f"Epoch accuracy on dev: {val_acc:.3f}")
    print(f"Epoch f1 on dev: {val_f1:.3f}")

print("Done!")

Adjusting learning rate of group 0 to 2.0000e-07.
Epoch: 0 of 100



train batches: 100%|██████████| 189/189 [01:17<00:00,  2.44it/s, loss: 2.110]


Adjusting learning rate of group 0 to 3.8000e-07.
Average epoch loss: 2.5900415816004316
Saved model to: /Users/johnsonzhou/git/comp90042-project/result/models/model_04_roberta_base_3_class.pth


dev batches: 100%|██████████| 23/23 [00:04<00:00,  5.66it/s,  acc: 0.472 f1: 0.472]


Epoch accuracy on dev: 0.471
Epoch f1 on dev: 0.471
Epoch: 1 of 100



...

Epoch accuracy on dev: 0.517
Epoch f1 on dev: 0.517
Epoch: 99 of 100



train batches: 100%|██████████| 189/189 [01:07<00:00,  2.82it/s, loss: 1.655]


Adjusting learning rate of group 0 to 2.0000e-06.
Average epoch loss: 1.9458723172308907


dev batches: 100%|██████████| 23/23 [00:02<00:00,  9.81it/s,  acc: 0.517 f1: 0.517]

Epoch accuracy on dev: 0.517
Epoch f1 on dev: 0.517
Done!
